In [ ]:
import numpy as np
import pandas as pd
from glob import glob


df = pd.concat([pd.read_hdf(f) for f in glob('test_gaps*.hdf')])
df = pd.read_hdf('test_gaps_0.hdf')
df['direction'] = df.apply(lambda x: np.argmax([x.B_x, x.B_y, x.B_z]), axis=1)
df['B'] = df.apply(lambda x: np.linalg.norm([x.B_x, x.B_y, x.B_z]), axis=1)
print(len(df))

vals_cols = [col for col in df.columns if len(df[col].unique()) > 1 and col!='E_gap']
print(vals_cols)
df.sort_values(vals_cols, inplace=True)
df.reset_index(drop=True, inplace=True)
df.to_hdf('gaps.hdf', 'all_data', complevel=9, complib='zlib')

In [ ]:
import holoviews as hv
hv.notebook_extension()

In [ ]:
%%opts Image [colorbar=True]
params = ['alpha', 'g', 'direction', 'orbital']
gb = df.groupby(params)
hm = {}
for i, gr in gb:
    data = gr.pivot_table(values=['E_gap'], index=['B'], columns=['V'])
    hm[i] = hv.Image(data, kdims=['V', 'B'], vdims=['E_gap'])
hv.HoloMap(hm, kdims=params)

In [ ]:
da = hv.Dataset(df.set_index(['alpha', 'g', 'direction', 'orbital']).to_xarray())
ds = hv.Dataset(da)